In [2]:
!pip install torch torchvision
import torch
from torch import nn
from torch import optim
from PIL import Image
from torchvision import transforms
from torchvision import models
from torchvision.utils import save_image

Defaulting to user installation because normal site-packages is not writeable
  Using cached torch-2.6.0-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.21.0-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 0.8/204.1 MB 6.7 MB/s eta 0:00:31
   ---------------------------------------- 1.8/204.1 MB 5.9 MB/s eta 0:00:35
    --------------------------------------- 3.1/204.1 MB 5.6 MB/s eta 0:00:36
    --------------------------------------- 3.7/204.1 MB 5.2 MB/s eta 0:00:39
    --------------------------------------- 4.5/204.1 MB 4.6 MB/s eta 0:00:44
   - -------------------------------------- 5.2/204.1 MB 4.4 MB/s eta 0:00:45
   - -------------------------------------- 6.3/204.1 MB 4.5 MB/s eta 0:00:44
   - -------------------------------------- 7.3/204.1 MB 4.6 MB/s eta 0:00:43
   - ------------

In [3]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.chosen_features = [0, 5, 10, 19, 28]
        self.model = models.vgg19(pretrained=True).features[:29]
    def forward(self, x):
        features = []
        for i, layer in enumerate(self.model):
            x = layer(x)
            if i in self.chosen_features:
                features.append(x)
        return features

In [4]:
def load_image(image_path, transform=None):
    image = Image.open(image_path)
    image = loader(image).unsqueeze(0)
    return image.to(device)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
loader = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()])

In [7]:
original_image = load_image("C:\\Users\\Tanishka\\OneDrive\\Pictures\\k.jpg")
style_image = load_image("C:\\Users\\Tanishka\\OneDrive\\Pictures\\style image.jpg")
generated_image = original_image.clone().requires_grad_(True).to(device)
vgg = VGG().to(device)
vgg.eval()
total_steps = 1500
lr = 1e-3
alpha = 10
beta = 0.5
optimizer = optim.Adam([generated_image], lr=lr)


C:\Users\Tanishka\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Tanishka\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
for step in range(total_steps):
    optimizer.zero_grad()
    generated_features = vgg(generated_image)
    original_features = vgg(original_image)
    style_features = vgg(style_image)
    style_loss = original_loss = 0
    for generated_feature, original_feature, style_feature in zip(generated_features, original_features, style_features):
        b, c, h, w = generated_feature.shape
        style_loss += torch.mean((generated_feature - style_feature) ** 2)
        original_loss += torch.mean((generated_feature - original_feature) ** 2)
    style_loss *= alpha / b
    original_loss *= beta / b
    loss = style_loss + original_loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if step % 200 == 0:
        print(f'Step [{step}/{total_steps}], Loss: {loss.item():.4f}')
        save_image(generated_image, f'generated_image_{step}.jpg')

Step [0/1500], Loss: 216.5932
